In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import AdaBoostRegressor

%matplotlib inline

In [2]:
names = ['API','Surf_X','Surf_Y','Date_Drilling','Date_Completion','Date_Production','Lateral_Length','Depth_TVD_PPLS','Erosion_PPLS','Pressure_PPLS','TOC_PPLS','Vcarb_PPLS','Vsand_PPLS','Vclay_PPLS','PR_PPLS','YM_PPLS','RHOB_PPLS','Res_PPLS','GR_PPLS','DT_PPLS','DTs_PPLS','Temperature','Temp_Anomaly','S3Tect_PPLS','S3_contrast_PPLS','Heat_Flow','Zone','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing','GasCum360','OilCum360']

df_data = pd.read_csv('./TrainSample.csv', 
                        header = None, 
                        sep = ';',
                        decimal = ',',
                        names = names,
                        skiprows = 1,
                        index_col=0,
                        parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                        dtype = {col: np.float32 for col in names}
                       )

df_test = pd.read_csv('./TestSample.csv',
                      header = None, 
                      sep = ';', 
                      decimal = ',', 
                      names = names,
                      skiprows = 1,
                      index_col=0,
                      parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                      dtype = {col: np.float32 for col in names}
                     )



In [3]:
## Outliers

df_data.loc[746, 'Frac_Gradient'] = np.nan
df_data.loc[553, 'ISIP'] = np.nan
df_data.loc[681, 'Min_Rate_Slurry'] = np.nan
df_data.loc[424, 'Res_PPLS'] = np.nan
df_data.loc[456, 'ShutInPressure_Fil'] = np.nan
df_data.loc[723, 'Min_Treating_Pressure'] = np.nan
df_data.loc[205, 'Min_Treating_Pressure'] = np.nan

In [4]:
data_mean_gas = df_data[['GasCum360']].mean()
data_mean_oil = df_data[['OilCum360']].mean()

In [5]:
##Définition des labels

y_gas = {'GasCum360' : df_data['GasCum360']}
y_oil = {'OilCum360' : df_data['OilCum360']}

df_target_gas = pd.DataFrame(y_gas)
df_target_oil = pd.DataFrame(y_oil)

In [6]:
cols_date = ['Date_Drilling','Date_Completion','Date_Production', 'GasCum360', 'OilCum360']

df_data.drop(cols_date, 1, inplace=True)
df_test.drop(cols_date, 1, inplace=True)

In [7]:
df_data.dropna()
df_test.dropna()
df_data = df_data.fillna(df_data.mean())
df_test = df_test.fillna(df_data.mean())

In [8]:
from sklearn import cross_validation

X_train, X_test, y_train_gas, y_test_gas = cross_validation.train_test_split(df_data, df_target_gas, test_size=0.2, random_state=0)
X_train, X_test, y_train_oil, y_test_oil = cross_validation.train_test_split(df_data, df_target_oil, test_size=0.2, random_state=0)

C:\Users\COCHE\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [31]:
## Algorithme - Choix des paramètres


from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

# Build classifier

regr_gas = AdaBoostRegressor()
regr_oil = AdaBoostRegressor()



In [32]:
# Specify parameters and distributions to sample form

from sklearn.model_selection import GridSearchCV

n_iter = 50

param_dist_oil = {
    "n_estimators": [x for x in range(40, 60, 1)],
    "learning_rate": [0.16],
    "loss": ['exponential']
}
clf_oil = GridSearchCV(regr_oil, param_grid=param_dist_oil, n_jobs=n_iter)

param_dist_gas = {
    "n_estimators": [41],
    "learning_rate": [0.23],
    "loss": ['exponential']
}

clf_gas = GridSearchCV(regr_gas, param_grid=param_dist_gas, n_jobs=n_iter)

In [33]:
clf_oil.fit(X_train, y_train_oil.values)
y_predict_oil = clf_oil.predict(X_test)

In [34]:
## Metrics training gas 

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

retour = 'Erreur Moyenne absolue : {}\n\
Erreur Moyenne carré : {}\n\
R2 : {}'.format(mean_absolute_error(y_test_gas, y_predict_gas),
                mean_squared_error(y_test_gas, y_predict_gas),
                r2_score(y_test_gas, y_predict_gas),
               )

print(retour)

Erreur Moyenne absolue : 0.43748671347
Erreur Moyenne carré : 0.309868346042
R2 : 0.603908590843


In [35]:
## Metrics training oil

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

retour = 'Erreur Moyenne absolue : {}\n\
Erreur Moyenne carré : {}\n\
R2 : {}'.format(mean_absolute_error(y_test_oil, y_predict_oil),
                mean_squared_error(y_test_oil, y_predict_oil),
                r2_score(y_test_oil, y_predict_oil)               
               )

print(retour)

Erreur Moyenne absolue : 0.378223308329
Erreur Moyenne carré : 0.263470389569
R2 : 0.69143147433


In [37]:
clf_oil.best_params_

{'learning_rate': 0.16, 'loss': 'exponential', 'n_estimators': 54}

In [19]:
## Fit regression model test data

clf_gas.fit(df_data, df_target_gas)
clf_oil.fit(df_data, df_target_oil)

y_predictFin_gas = clf_gas.predict(df_test)
y_predictFin_oil = clf_oil.predict(df_test)

In [26]:
gascumpred = []
oilcumpred = []

gascumpred = [element for element in y_predictFin_gas]
oilcumpred = [element for element in y_predictFin_oil]

gascumpred_mean = mean_absolute_error(y_test_gas, y_predict_gas)
oilcumpred_mean = mean_absolute_error(y_test_oil, y_predict_oil)

GasCum360_inf = gascumpred - abs(gascumpred_mean)
GasCum360_sup = gascumpred + abs(gascumpred_mean)

OilCum360_inf = oilcumpred - abs(oilcumpred_mean)
OilCum360_sup = oilcumpred + abs(oilcumpred_mean)

GasCum360_inf.tolist
GasCum360_sup.tolist

OilCum360_inf.tolist
OilCum360_sup.tolist

<function tolist>

In [27]:
y_predict_gas[:10]

array([-0.46662758,  0.99059787, -0.44271366, -0.63296575, -0.60025688,
        0.17840302,  0.17716106, -0.5188524 , -0.38211596, -0.36759049])

In [28]:
y_test_gas.head(10)

,GasCum360
API,
521.0,-1.056089
38.0,-0.036146
471.0,-0.150183
81.0,-0.798361
237.0,-0.990523
442.0,-0.842177
375.0,0.419826
610.0,-0.879051
706.0,-0.338057


In [29]:
## Output

id_test = df_test.index.tolist()

output = pd.DataFrame({'API': id_test,
                       'GasCum360_INF': GasCum360_inf,
                       'GasCum360_SUP': GasCum360_sup,
                       'OilCum360_INF': OilCum360_inf,
                       'OilCum360_SUP': OilCum360_sup},
                      index=id_test
                     )

output.head()

output.to_csv('coche-julien-challenge-total.csv', index=False, sep= ';', decimal=',')

In [30]:
print(output.head(5))

         API  GasCum360_INF  GasCum360_SUP  OilCum360_INF  OilCum360_SUP
633.0  633.0      -0.999684      -0.124711       0.084682       0.840552
587.0  587.0       0.574497       1.449471      -1.124173      -0.368303
264.0  264.0       0.403615       1.278589      -1.093547      -0.337676
680.0  680.0       0.219419       1.094392      -1.123814      -0.367944
443.0  443.0       0.116703       0.991676      -0.257412       0.498459
